WIP

1. Go to https://discord.com/developers/applications
2. Click "New Application" and give it a name (I called mine sec-bot)
3. Create a token and copy it 
4. Under "Privileged Gateway Intents", enable "Message Content Intent"
5. Go to Oauth2. Enable bot, messages.read and Send Messages